In [1]:
import numpy
print(numpy.__version__)
print(numpy._core.multiarray._reconstruct)  # ← this line should succeed

import accelerate
print("Accelerate version:", accelerate.__version__)

1.25.2


AttributeError: module 'numpy' has no attribute '_core'

In [ ]:
import re
import json
from datasets import load_dataset
from pathlib import Path

import numpy
print("Numpy version:", numpy.__version__)  # should be 1.26.4

import accelerate
print("Accelerate version:", accelerate.__version__)

import transformers
print("Transformers version:",transformers.__version__)  # Should be 4.53.0 or higher

import torch
print("Torch version:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())

Numpy version: 1.26.4


AttributeError: module 'numpy._core' has no attribute 'multiarray'

In [ ]:
import torch
print("Torch version:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

Torch version: 2.7.1
MPS available: True
MPS built: True


In [ ]:
import accelerate
import transformers
print(accelerate.__version__)
print(transformers.__version__)


ModuleNotFoundError: No module named 'transformers'

# Start from here

In [2]:
import re
import json
from datasets import load_dataset
from pathlib import Path

In [3]:
dataset = load_dataset("openai/gsm8k", "main")

In [4]:
train_data = dataset["train"]

In [5]:
example = train_data[0]
print(example["question"])
print(example["answer"])


Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [6]:
train_data

Dataset({
    features: ['question', 'answer'],
    num_rows: 7473
})

## Error in the final answer only

In [7]:
question = example["question"]
original_answer = example["answer"]

# Split into lines for L1, L2, ...
lines = original_answer.strip().splitlines()

# Find the line that contains '#### <number>'
for i, line in enumerate(lines):
    match = re.match(r"####\s*(-?\d+)", line.strip())
    if match:
        correct = int(match.group(1))
        flawed = correct + 1
        error_line_index = i
        break
else:
    print("❌ No final answer line found.")
    exit()

# Create modified lines
flawed_lines = lines.copy()
flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index])

# Recombine into modified answer
flawed_answer = "\n".join(flawed_lines)

# Compute error location as "L<N>"
error_location = f"L{error_line_index + 1}"

# Create label
label = {
    "verdict": "Flawed",
    "error_details": {
        "error_type": "computational_error",
        "erroneous_line_number": error_location,
        "explanation": f"The final answer is too high by 1. It should be {correct}, not {flawed}.",
        "error_in_text": lines[error_line_index],
        "correction_in_text": f"#### {correct}"
    }
}

# Output
print("📌 QUESTION:\n", question)
print("\n✅ ORIGINAL ANSWER:\n", original_answer)
print("\n❌ FLAWED ANSWER:\n", flawed_answer)
print("\n🧾 LABEL:\n", label)


📌 QUESTION:
 Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

✅ ORIGINAL ANSWER:
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

❌ FLAWED ANSWER:
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 73

🧾 LABEL:
 {'verdict': 'Flawed', 'error_details': {'error_type': 'computational_error', 'erroneous_line_number': 'L3', 'explanation': 'The final answer is too high by 1. It should be 72, not 73.', 'error_in_text': '#### 72', 'correction_in_text': '#### 72'}}


In [8]:
# Output path
output_path = Path("gsm8k_train_flawed_plus1_final_answer.jsonl")

with output_path.open("w", encoding="utf-8") as f_out:
    for i, ex in enumerate(dataset["train"]):
        q = ex["question"]
        a = ex["answer"]

        # Split into lines for L1, L2, ...
        lines = a.strip().splitlines()

        # Find the line with the final answer
        for idx, line in enumerate(lines):
            match = re.match(r"####\s*(-?\d+)", line.strip())
            if match:
                correct = int(match.group(1))
                flawed = correct + 1
                error_line_index = idx
                break
        else:
            print(i, "❌ No final answer line found.\n")
            print(a)
            continue  # skip if no final answer found

        # Copy and modify lines
        flawed_lines = lines.copy()
        flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index]) #f"#### {flawed}"

        # # Replace the final display number in any inline computation
        # flawed_lines = [
        #     re.sub(r"(>>)" + str(correct) + r"\b", r"\1" + str(flawed), line)
        #     for line in flawed_lines
        # ]

        flawed_answer = "\n".join(flawed_lines)

        error_location = f"L{error_line_index + 1}"

        # Create label
        label = {
            "verdict": "Flawed",
            "error_details": {
                "error_type": "computational_error",
                "erroneous_line_number": f"L{error_line_index + 1}",
                "explanation": f"The final answer is too high by 1. It should be {correct}, not {flawed}.",
                "error_in_text": lines[error_line_index],
                "correction_in_text": f"#### {correct}"
            }
        }

        # Final example
        entry = {
            "id": i,
            "question": q,
            "flawed_answer": flawed_answer,
            "label": label
        }
        f_out.write(json.dumps(entry) + "\n")
        
print(f"✅ Finished writing {output_path.name}")

✅ Finished writing gsm8k_train_flawed_plus1_final_answer.jsonl


In [9]:
with open("gsm8k_train_flawed_plus1_final_answer.jsonl") as f:
    print(len(f.readlines()))  # Should be >0


7473


In [10]:
# Output path
output_path = Path("gsm8k_test_flawed_plus1_final_answer.jsonl")

with output_path.open("w", encoding="utf-8") as f_out:
    for i, ex in enumerate(dataset["test"]):
        q = ex["question"]
        a = ex["answer"]

        # Split into lines for L1, L2, ...
        lines = a.strip().splitlines()

        # Find the line with the final answer
        for idx, line in enumerate(lines):
            match = re.match(r"####\s*(-?\d+)", line.strip())
            if match:
                correct = int(match.group(1))
                flawed = correct + 1
                error_line_index = idx
                break
        else:
            print(i, "❌ No final answer line found.\n")
            print(a)
            continue  # skip if no final answer found

        # Copy and modify lines
        flawed_lines = lines.copy()
        flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index]) #f"#### {flawed}"

        # # Replace the final display number in any inline computation
        # flawed_lines = [
        #     re.sub(r"(>>)" + str(correct) + r"\b", r"\1" + str(flawed), line)
        #     for line in flawed_lines
        # ]

        flawed_answer = "\n".join(flawed_lines)

        error_location = f"L{error_line_index + 1}"

        # Create label
        label = {
            "verdict": "Flawed",
            "error_details": {
                "error_type": "computational_error",
                "erroneous_line_number": f"L{error_line_index + 1}",
                "explanation": f"The final answer is too high by 1. It should be {correct}, not {flawed}.",
                "error_in_text": lines[error_line_index],
                "correction_in_text": f"#### {correct}"
            }
        }

        # Final example
        entry = {
            "id": i,
            "question": q,
            "flawed_answer": flawed_answer,
            "label": label
        }
        f_out.write(json.dumps(entry) + "\n")

print(f"✅ Finished writing {output_path.name}")

✅ Finished writing gsm8k_test_flawed_plus1_final_answer.jsonl


## Error in the second last line

In [11]:
question = example["question"]
original_answer = example["answer"]

# Split into lines for L1, L2, ...
lines = original_answer.strip().splitlines()

# Find the line that contains '#### <number>'
for i, line in enumerate(lines):
    match = re.match(r"####\s*(-?\d+)", line.strip())
    if match:
        correct = int(match.group(1))
        flawed = correct + 1
        error_line_index = i
        break
else:
    print("❌ No final answer line found.")
    exit()

# Create modified lines
flawed_lines = lines.copy()
flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index])

# Replace only the number after the final '>>' (the human-readable result)
flawed_lines = [
    re.sub(str(correct) + r"(>>)" + str(correct) + r"\b", str(flawed) + r">>" + str(flawed), line)
    for line in flawed_lines
]

# Recombine into modified answer
flawed_answer = "\n".join(flawed_lines)

# Compute error location as "L<N>"
error_location = f"L{error_line_index}"

# Create label
label = {
    "verdict": "Flawed",
    "error_details": {
        "error_type": "computational_error",
        "erroneous_line_number": error_location,
        "explanation": f"The answer is too high by 1. It should be {correct}, not {flawed}.",
        "error_in_text": lines[error_line_index],
        "correction_in_text": f"#### {correct}"
    }
}

# Output
print("📌 QUESTION:\n", question)
print("\n✅ ORIGINAL ANSWER:\n", original_answer)
print("\n❌ FLAWED ANSWER:\n", flawed_answer)
print("\n🧾 LABEL:\n", label)


📌 QUESTION:
 Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

✅ ORIGINAL ANSWER:
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72

❌ FLAWED ANSWER:
 Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=73>>73 clips altogether in April and May.
#### 73

🧾 LABEL:
 {'verdict': 'Flawed', 'error_details': {'error_type': 'computational_error', 'erroneous_line_number': 'L2', 'explanation': 'The answer is too high by 1. It should be 72, not 73.', 'error_in_text': '#### 72', 'correction_in_text': '#### 72'}}


In [12]:
# Output path
output_path = Path("gsm8k_train_flawed_plus1_2nd_last.jsonl")

with output_path.open("w", encoding="utf-8") as f_out:
    for i, ex in enumerate(dataset["train"]):
        q = ex["question"]
        a = ex["answer"]

        # Split into lines for L1, L2, ...
        lines = a.strip().splitlines()

        # Find the line with the final answer
        for idx, line in enumerate(lines):
            match = re.match(r"####\s*(-?\d+)", line.strip())
            if match:
                correct = int(match.group(1))
                flawed = correct + 1
                error_line_index = idx
                break
        else:
            print(i, "❌ No final answer line found.\n")
            print(a)
            continue  # skip if no final answer found

        # Copy and modify lines
        flawed_lines = lines.copy()
        flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index]) #f"#### {flawed}"

        # Replace only the number after the final '>>' (the human-readable result)
        flawed_lines = [
            re.sub(str(correct) + r"(>>)" + str(correct) + r"\b", str(flawed) + r">>" + str(flawed), line)
            for line in flawed_lines
        ]

        flawed_answer = "\n".join(flawed_lines)

        error_location = f"L{error_line_index}"

        # Create label
        label = {
            "verdict": "Flawed",
            "error_details": {
                "error_type": "computational_error",
                "erroneous_line_number": f"L{error_line_index}",
                "explanation": f"The 2n last line value is too high by 1. It should be {correct}, not {flawed}.",
                "error_in_text": lines[error_line_index],
                "correction_in_text": f"#### {correct}"
            }
        }

        # Final example
        entry = {
            "id": i,
            "question": q,
            "flawed_answer": flawed_answer,
            "label": label
        }
        f_out.write(json.dumps(entry) + "\n")

print(f"✅ Finished writing {output_path.name}")

✅ Finished writing gsm8k_train_flawed_plus1_2nd_last.jsonl


In [13]:
# Output path
output_path = Path("gsm8k_test_flawed_plus1_2nd_last.jsonl")

with output_path.open("w", encoding="utf-8") as f_out:
    for i, ex in enumerate(dataset["test"]):
        q = ex["question"]
        a = ex["answer"]

        # Split into lines for L1, L2, ...
        lines = a.strip().splitlines()

        # Find the line with the final answer
        for idx, line in enumerate(lines):
            match = re.match(r"####\s*(-?\d+)", line.strip())
            if match:
                correct = int(match.group(1))
                flawed = correct + 1
                error_line_index = idx
                break
        else:
            print(i, "❌ No final answer line found.\n")
            print(a)
            continue  # skip if no final answer found

        # Copy and modify lines
        flawed_lines = lines.copy()
        flawed_lines[error_line_index] = re.sub(r"####\s*\d+", f"#### {flawed}", lines[error_line_index]) #f"#### {flawed}"

        # Replace only the number after the final '>>' (the human-readable result)
        flawed_lines = [
            re.sub(str(correct) + r"(>>)" + str(correct) + r"\b", str(flawed) + r">>" + str(flawed), line)
            for line in flawed_lines
        ]

        flawed_answer = "\n".join(flawed_lines)

        error_location = f"L{error_line_index}"

        # Create label
        label = {
            "verdict": "Flawed",
            "error_details": {
                "error_type": "computational_error",
                "erroneous_line_number": f"L{error_line_index}",
                "explanation": f"The 2n last line value is too high by 1. It should be {correct}, not {flawed}.",
                "error_in_text": lines[error_line_index],
                "correction_in_text": f"#### {correct}"
            }
        }

        # Final example
        entry = {
            "id": i,
            "question": q,
            "flawed_answer": flawed_answer,
            "label": label
        }
        f_out.write(json.dumps(entry) + "\n")

print(f"✅ Finished writing {output_path.name}")

✅ Finished writing gsm8k_test_flawed_plus1_2nd_last.jsonl


# Model

## Combine data

In [14]:
with open("gsm8k_train_flawed_plus1_final_answer.jsonl", "r") as f:
    lines = f.readlines()
    print(f"Total lines: {len(lines)}")
    print("First line:", lines[0] if lines else "No data")


Total lines: 7473
First line: {"id": 0, "question": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?", "flawed_answer": "Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 73", "label": {"verdict": "Flawed", "error_details": {"error_type": "computational_error", "erroneous_line_number": "L3", "explanation": "The final answer is too high by 1. It should be 72, not 73.", "error_in_text": "#### 72", "correction_in_text": "#### 72"}}}



In [15]:
original_dataset = load_dataset("openai/gsm8k", "main")

def load_jsonl(path):
    with open(path, 'r') as f:
        return [json.loads(line) for line in f]

flawed_train = load_jsonl("gsm8k_train_flawed_plus1_final_answer.jsonl")
flawed_test = load_jsonl("gsm8k_test_flawed_plus1_final_answer.jsonl")

combined_train = []

# Add correct examples from original dataset
for ex in original_dataset["train"]:
    combined_train.append({
        "question": ex["question"],
        "solution": ex["answer"],
        "label": "Correct"
    })

# Add flawed examples
for ex in flawed_train:
    combined_train.append({
        "question": ex["question"],
        "solution": ex["flawed_answer"],
        "label": "Flawed"
    })

print(f"Combined training set size: {len(combined_train)}")


Combined training set size: 14946


In [16]:
combined_test = []

# Add correct examples from original test split
for ex in original_dataset["test"]:
    combined_test.append({
        "question": ex["question"],
        "solution": ex["answer"],
        "label": "Correct"
    })

# Add flawed examples from test JSONL
for ex in flawed_test:
    combined_test.append({
        "question": ex["question"],
        "solution": ex["flawed_answer"],
        "label": "Flawed"
    })

print(f"Combined test set size: {len(combined_test)}")


Combined test set size: 2638


In [17]:
# Convert to HUgging Face Dataset
from datasets import Dataset
train_dataset = Dataset.from_list(combined_train)
test_dataset = Dataset.from_list(combined_test)

In [18]:
import torch
print("Torch version:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())

Torch version: 2.5.1
MPS available: True


In [19]:
# Preprocessing with tokenizer
import torch
from transformers import AutoTokenizer

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(example):
    full_input = f"Question:\n{example['question']}\n\nSolution:\n{example['solution']}"
    return tokenizer(full_input, truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(tokenize_fn)
test_dataset = test_dataset.map(tokenize_fn)


Map:   0%|          | 0/14946 [00:00<?, ? examples/s]

Map:   0%|          | 0/2638 [00:00<?, ? examples/s]

In [20]:
dir(train_dataset)

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contigu

In [22]:
train_dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'solution': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
 'label': 'Correct',
 'input_ids': [101,
  3160,
  1024,
  21521,
  2853,
  15281,
  2000,
  4466,
  1997,
  2014,
  2814,
  1999,
  2258,
  1010,
  1998,
  2059,
  2016,
  2853,
  2431,
  2004,
  2116,
  15281,
  1999,
  2089,
  1012,
  2129,
  2116,
  15281,
  2106,
  21521,
  5271,
  10462,
  1999,
  2258,
  1998,
  2089,
  1029,
  5576,
  1024,
  21521,
  2853,
  4466,
  1013,
  1016,
  1027,
  1026,
  1026,
  4466,
  1013,
  1016,
  1027,
  2484,
  1028,
  1028,
  2484,
  15281,
  1999,
  2089,
  1012,
  21521,
  2853,
  4466,
  1009,
  2484,
  1027,
  1026,
  1026,
  4466,
  1009,
  2484,
  1027,
  5824,
  1028,
  1028,
  5824,
  15281,
  10462,
  1999,
  2258,
  1998,

#

In [ ]:
# Label encoding
label_map = {"Correct": 0, "Flawed": 1}
train_dataset = train_dataset.map(lambda e: {"labels": label_map[e["label"]]})
test_dataset = test_dataset.map(lambda e: {"labels": label_map[e["label"]]})

Map: 100%|██████████| 2638/2638 [00:00<00:00, 24672.43 examples/s]


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)

# Use a lightweight and stable model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./verifier_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()


ImportError: cannot import name 'TransformGetItemToIndex' from 'torch._dynamo._trace_wrapped_higher_order_op' (/opt/anaconda3/envs/gsm8k-verifier/lib/python3.10/site-packages/torch/_dynamo/_trace_wrapped_higher_order_op.py)

In [ ]:
import transformers
import accelerate

print(transformers.__version__)  # should be >= 4.38
print(accelerate.__version__)   # should be >= 0.26.0


4.53.0
1.8.0
